In [1]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn import preprocessing

In [2]:
'''
ID - an Id that represents a (Shop, Item) tuple within the test set
shop_id - unique identifier of a shop
item_id - unique identifier of a product
item_category_id - unique identifier of item category
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
item_price - current price of an item
date - date in format dd/mm/yyyy
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item
shop_name - name of shop
item_category_name - name of item category
'''

'\nID - an Id that represents a (Shop, Item) tuple within the test set\nshop_id - unique identifier of a shop\nitem_id - unique identifier of a product\nitem_category_id - unique identifier of item category\nitem_cnt_day - number of products sold. You are predicting a monthly amount of this measure\nitem_price - current price of an item\ndate - date in format dd/mm/yyyy\ndate_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33\nitem_name - name of item\nshop_name - name of shop\nitem_category_name - name of item category\n'

In [3]:
items = pd.read_csv("../data/items.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
train = pd.read_csv("../data/sales_train.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
test = pd.read_csv("../data/test.csv")

In [4]:
print('items : ', len(items))
print('item_cat : ', len(item_cat))
print('train : ' , len(train))
print('test : ', len(test))
print('shops : ', len(shops))

items :  22170
item_cat :  84
train :  2935849
test :  214200
shops :  60


In [5]:
print(min(train['item_cnt_day']), max(train['item_cnt_day']))

-22.0 2169.0


In [6]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
train.head(10)
# train.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [9]:
train['date'] = train['date'].str[3:]

In [10]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,01.2013,0,59,22154,999.00,1.0
1,01.2013,0,25,2552,899.00,1.0
2,01.2013,0,25,2552,899.00,-1.0
3,01.2013,0,25,2554,1709.05,1.0
4,01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.2015,33,25,7409,299.00,1.0
2935845,10.2015,33,25,7460,299.00,1.0
2935846,10.2015,33,25,7459,349.00,1.0
2935847,10.2015,33,25,7440,299.00,1.0


In [11]:
train_groupby = train.groupby(['shop_id', 'item_id', 'date'])['item_cnt_day'].agg('sum').reset_index()
train_groupby

,shop_id,item_id,date,item_cnt_day
0,0,30,02.2013,31.0
1,0,31,02.2013,11.0
2,0,32,01.2013,6.0
3,0,32,02.2013,10.0
4,0,33,01.2013,3.0
...,...,...,...,...
1609119,59,22164,04.2015,2.0
1609120,59,22164,07.2015,1.0
1609121,59,22167,06.2014,1.0
1609122,59,22167,10.2013,1.0


In [14]:
train_groupby_shop = train_groupby.groupby(['shop_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_item = train_groupby.groupby(['item_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_shop_and_item = train_groupby.groupby(['shop_id', 'item_id','date'])['item_cnt_day'].agg('sum').reset_index()

In [15]:
#train_groupby_item[train_groupby_item['item_id'] == 2552]
train_groupby_item[train_groupby['item_id'] == 2552]

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,item_id,item_cnt_day


In [16]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [17]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [18]:
test.head() ##언젠지 없냐?

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [19]:
min(items['item_category_id'])

0

In [20]:
train_groupby_shop.head()

,shop_id,item_cnt_day
0,0,11705.0
1,1,6311.0
2,2,30620.0
3,3,28355.0
4,4,43942.0


In [21]:
train_groupby_shop = train_groupby_shop.rename(columns = {'item_cnt_day': '#2_amount'}, inplace = False)
train = pd.merge(train, train_groupby_shop, on='shop_id')

In [22]:
train['tmp'] = train['item_price']*train['item_cnt_day']
train_groupby_shop_price = train.groupby(['shop_id'])['tmp'].agg('sum').reset_index()
train_groupby_shop_price = train_groupby_shop_price.rename(columns = {'tmp': '#2_price'}, inplace = False)
train = pd.merge(train, train_groupby_shop_price, on ='shop_id')

In [23]:
train = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

In [24]:
train_groupby_item_category = train.groupby(['item_category_id'])['item_cnt_day'].agg('sum').reset_index()

In [25]:
train = pd.merge(train, train_groupby_item_category , on='item_category_id')

In [26]:
train = train.rename(columns = {'item_cnt_day_y': '#3_amount'}, inplace = False)

In [27]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount
0,01.2013,0,59,22154,999.0,1.0,48993.0,999.0,4.522658e+07,37,203284.0
1,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
2,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
3,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
4,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,63388.0,250.0,4.979206e+07,36,22.0
2935845,09.2015,32,55,13091,1000.0,1.0,63388.0,1000.0,4.979206e+07,36,22.0
2935846,09.2015,32,55,13094,2500.0,1.0,63388.0,2500.0,4.979206e+07,36,22.0
2935847,09.2015,32,55,13094,2500.0,2.0,63388.0,5000.0,4.979206e+07,36,22.0


In [28]:
train_groupby_item = train_groupby_item.rename(columns = {'item_cnt_day' : '#4_amount'}, inplace = False)

In [29]:
train = pd.merge(train, train_groupby_item , on='item_id')

In [30]:
#train = train.drop(['item_cnt_day_x'], axis = 1)

In [31]:
train['month'] = train['date'].str[:2]

In [32]:
train['year'] = train['date'].str[3:]

In [33]:
def normalize_column(df, column_name):
    x = df[[column_name]]
    # Create x, where x the 'scores' column's values as floats
    x = x.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled, columns = [column_name])
    
    df[column_name] = df_normalized
    
    return df

In [34]:
normalize_column(train, '#2_amount')
normalize_column(train, '#2_price')
normalize_column(train, '#3_amount')
normalize_column(train, '#4_amount')
train['month'] = train['month'].astype(int)
train['year'] = train['year'].astype(int)

In [35]:
train.drop('tmp', axis = 1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,#2_price,item_category_id,#3_amount,#4_amount,month,year
0,01.2013,0,59,22154,999.0,1.0,0.156751,0.190977,37,0.320550,0.000373,1,2013
1,01.2013,0,25,22154,999.0,1.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
2,01.2013,0,25,22154,999.0,1.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
3,01.2013,0,25,22154,999.0,1.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
4,01.2013,0,25,22154,999.0,1.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,0.210418,36,0.000033,0.000123,10,2015
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,0.210418,36,0.000033,0.000064,9,2015
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,0.210418,36,0.000033,0.000075,9,2015
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,0.210418,36,0.000033,0.000075,9,2015


# 6.1

In [36]:
train['item_price_avg'] = train['item_price'].mean()

# 6.2

In [37]:
out = []
unique_cat = train['item_category_id'].unique()
for cat in unique_cat:
    same_cat_diff_shop = train[train['item_category_id']==cat]
    same_shop = same_cat_diff_shop.drop_duplicates(subset='shop_id', keep="last") #unique shops
    norm=(same_shop['item_price']-same_shop['item_price'].mean())/same_shop['item_price'].std()
    same_shop['item_cat_price_norm'] = norm
    arr = np.array(same_shop)[:,[2,8, -1]]
    out.append(arr)
    #out.append(same_shop['date','item_category_id' ,'item_cat_price_norm'])
    #train = train.merge(same_shop, on = list(train.columns), how = 'left')

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [38]:
out = np.array(out)
v = np.vstack(out)
df = pd.DataFrame(v, columns = ['shop_id', 'item_category_id', 'item_cat_price_norm'])
df = df.fillna(0)
df_train = train.merge(df, on = ['shop_id', 'item_category_id'], how = 'left')
train = df_train.drop(['item_cat_price_norm_x'], axis = 1)

/Users/emphymacbook/opt/anaconda3/envs/textmining/lib/python3.7/site-packages/pandas/core/reshape/merge.py:1127: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


KeyError: "['item_cat_price_norm_x'] not found in axis"

In [ ]:
train

# 1

In [ ]:
train = pd.concat([train,pd.get_dummies(train['shop_id'], prefix='shop_id')],axis=1)

In [39]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,0.320550,0.000373,1,2013,890.853233
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,0.000033,0.000123,10,2015,890.853233
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,0.000033,0.000064,9,2015,890.853233
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,0.000033,0.000075,9,2015,890.853233
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,0.000033,0.000075,9,2015,890.853233


## 8

In [87]:
lag2015 = train[train['date'] == '10.2015']['item_cnt_day_x']
lag2013 = train[train['date'] == '11.2013']['item_cnt_day_x']
lag2014 = train[train['date'] == '11.2014']['item_cnt_day_x']

In [97]:
lag2013.shape, lag2014.shape, lag2015.shape

((96736,), (86428,), (53514,))

In [102]:
lag2013_ = lag2013.to_frame()
lag2014_ = lag2014.to_frame()
lag2015_ = lag2015.to_frame()

temp2 = pd.concat([lag2013_, lag2014_,lag2015_], axis=1).fillna(0)
temp2.columns=['2013.11','2014.11','2015.10']
temp2

,2013.11,2014.11,2015.10
9,0.0,0.0,1.0
14,0.0,0.0,1.0
25,0.0,0.0,1.0
31,0.0,0.0,1.0
45,0.0,0.0,1.0
...,...,...,...
2935841,0.0,0.0,1.0
2935842,0.0,0.0,1.0
2935843,0.0,0.0,1.0
2935844,0.0,0.0,1.0


In [104]:
train = pd.concat([train, temp2], axis=1).fillna(0)
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,0.320550,0.000373,1,2013,890.853233,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,890.853233,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,0.000033,0.000123,10,2015,890.853233,0.0,0.0,1.0
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,0.000033,0.000064,9,2015,890.853233,0.0,0.0,0.0
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,0.000033,0.000075,9,2015,890.853233,0.0,0.0,0.0
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,0.000033,0.000075,9,2015,890.853233,0.0,0.0,0.0


## 9 

In [105]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [106]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,0.32055,0.000373,1,2013,890.853233,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,0.0,0.0,0.0


In [107]:
from sklearn.feature_extraction.text import CountVectorizer

item_name_list = items['item_name']

vect = CountVectorizer(analyzer='word', max_features=20)
bow_feature = vect.fit_transform(item_name_list).toarray()
bow_feature

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [108]:
type(bow_feature)

numpy.ndarray

In [109]:
bow_df = pd.DataFrame(bow_feature)
bow_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22165,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
22166,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
22167,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22168,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [110]:
item_ids = train['item_id']
item_ids

0          22154
1          22154
2          22154
3          22154
4          22154
           ...  
2935844    13093
2935845    13091
2935846    13094
2935847    13094
2935848    13092
Name: item_id, Length: 2935849, dtype: int64

In [111]:
temp = bow_df.iloc[item_ids].reset_index(drop=True)
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935845,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935846,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935847,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [112]:
train_ = pd.concat([train, temp], axis=1)
train_

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,...,10,11,12,13,14,15,16,17,18,19
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,0,0,0,0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
